In [21]:
import json

In [22]:
with open('C:/Users/Dominika/Downloads/predictionTasks(2).json', 'r') as file:
    ruleset = json.load(file)

In [23]:
not_prdicted = []
i = 0
for rule in ruleset:
    if not rule["candidates"]:
        i += 1
        not_prdicted.append(rule["predictionTask"]["c"]["s"]["localName"])
print(i)
print(len(not_prdicted))

578
578


In [24]:
import csv
with open('not_prdicted_1020.csv', 'w', newline='\n') as f:
     
    write = csv.writer(f)
     
    write.writerows([not_prdicted])

In [5]:
#!pip install rdflib
import rdflib
from rdflib import Graph
import pandas as pd
import json
import urllib.parse

In [6]:
g = Graph()
g.parse("kg-mic.ttl")

<Graph identifier=N7dd91cdb3b584cf7921a22164c18b830 (<class 'rdflib.graph.Graph'>)>

In [25]:

bioregistry_prefixes = {}
f = open("C:/Users/Dominika/Documents/SKOLA/ING/DIPLOMKA/registry.json")
data = json.load(f)
for entity in data.values():
    bioregistry_prefixes[entity["prefix"]] = {"name": entity["name"], "uri_format": entity["uri_format"]}
f.close()
del data



def extract_uri(id: str):
    def_prefix = bioregistry_prefixes.get("obo")
    uf = def_prefix["uri_format"]  # type: str
    return uf.replace("$1", id)


taxons_prefixed = []
for row in not_prdicted:
    uri_1 = extract_uri(row)
    taxons_prefixed.append(uri_1)

print(len(taxons_prefixed))

578


In [26]:
query = """
SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
}
"""

bad_rule_media = []

for row in taxons_prefixed:
    subjectitos = rdflib.term.URIRef(row)
    filtered_gq = g.query(query, initBindings = {
        "s": subjectitos,
        "p": rdflib.term.URIRef("https://w3id.org/biolink/vocab/occurs_in")
    })
    for row in filtered_gq:
        bad_rule_media.append(row.o)

print(len(bad_rule_media))



999


In [27]:
bad_media_dict = dict.fromkeys(bad_rule_media, 0)
for row in bad_rule_media:
    bad_media_dict[row] = bad_media_dict[row] + 1

In [28]:
data_sorted = {k: v for k, v in sorted(bad_media_dict.items(), key=lambda x: x[1], reverse=True)}
print(data_sorted)

{rdflib.term.URIRef('urn:unknown:medium%3A514'): 171, rdflib.term.URIRef('urn:unknown:medium%3A830'): 107, rdflib.term.URIRef('urn:unknown:medium%3A92'): 94, rdflib.term.URIRef('urn:unknown:medium%3A65'): 85, rdflib.term.URIRef('urn:unknown:medium%3A693'): 58, rdflib.term.URIRef('urn:unknown:medium%3AJ118'): 51, rdflib.term.URIRef('urn:unknown:medium%3AJ26'): 48, rdflib.term.URIRef('urn:unknown:medium%3A535'): 37, rdflib.term.URIRef('urn:unknown:medium%3AJ43'): 31, rdflib.term.URIRef('urn:unknown:medium%3A11'): 22, rdflib.term.URIRef('urn:unknown:medium%3AJ346'): 21, rdflib.term.URIRef('urn:unknown:medium%3A1'): 21, rdflib.term.URIRef('urn:unknown:medium%3A553'): 17, rdflib.term.URIRef('urn:unknown:medium%3AJ28'): 16, rdflib.term.URIRef('urn:unknown:medium%3AJ14'): 15, rdflib.term.URIRef('urn:unknown:medium%3A215'): 14, rdflib.term.URIRef('urn:unknown:medium%3A645'): 13, rdflib.term.URIRef('urn:unknown:medium%3A104'): 10, rdflib.term.URIRef('urn:unknown:medium%3AJ386'): 10, rdflib.term

In [29]:
data_sorted

{rdflib.term.URIRef('urn:unknown:medium%3A514'): 171,
 rdflib.term.URIRef('urn:unknown:medium%3A830'): 107,
 rdflib.term.URIRef('urn:unknown:medium%3A92'): 94,
 rdflib.term.URIRef('urn:unknown:medium%3A65'): 85,
 rdflib.term.URIRef('urn:unknown:medium%3A693'): 58,
 rdflib.term.URIRef('urn:unknown:medium%3AJ118'): 51,
 rdflib.term.URIRef('urn:unknown:medium%3AJ26'): 48,
 rdflib.term.URIRef('urn:unknown:medium%3A535'): 37,
 rdflib.term.URIRef('urn:unknown:medium%3AJ43'): 31,
 rdflib.term.URIRef('urn:unknown:medium%3A11'): 22,
 rdflib.term.URIRef('urn:unknown:medium%3AJ346'): 21,
 rdflib.term.URIRef('urn:unknown:medium%3A1'): 21,
 rdflib.term.URIRef('urn:unknown:medium%3A553'): 17,
 rdflib.term.URIRef('urn:unknown:medium%3AJ28'): 16,
 rdflib.term.URIRef('urn:unknown:medium%3AJ14'): 15,
 rdflib.term.URIRef('urn:unknown:medium%3A215'): 14,
 rdflib.term.URIRef('urn:unknown:medium%3A645'): 13,
 rdflib.term.URIRef('urn:unknown:medium%3A104'): 10,
 rdflib.term.URIRef('urn:unknown:medium%3AJ386'

In [30]:
import json
with open("bad_media_1020.json", "w") as outfile: 
    json.dump(data_sorted, outfile)